### <b>Deep Convolutional Neural Network for NLP/ sentiment analysis</b>

### <b>1. Importing/loading libraries</b>

In [67]:
import numpy as np
import pandas as pd
import math
import re
from bs4 import BeautifulSoup
import tensorflow as tf
import tensorflow_datasets as tfds

### <b>2. Data preprocessing</b>

In [68]:
columns = ["sentiment", "id", "date", "query", "user", "text"]

training_data = pd.read_csv("training_data.csv", header = None, names = columns, engine = "python", encoding = "latin1")
test_data = pd.read_csv("test_data.csv", header = None, names = columns, engine = "python", encoding = "latin1")

size_of_training_data = len(training_data)
size_of_test_data = len(test_data)

print(f"Size of training data: {size_of_training_data}")
print(f"Size of test data: {size_of_test_data}")
print(f"Training data records: \n{training_data.head(10)}\n")
print(f"Test data 10 records: \n{test_data.head(10)}\n")

Size of training data: 1992
Size of test data: 498
Training data records: 
   sentiment  id                          date    query          user  \
0          4   3  Mon May 11 03:17:40 UTC 2009  kindle2        tpryan   
1          4   4  Mon May 11 03:18:03 UTC 2009  kindle2        vcu451   
2          4   5  Mon May 11 03:18:54 UTC 2009  kindle2        chadfu   
3          4   6  Mon May 11 03:19:04 UTC 2009  kindle2         SIX15   
4          4   7  Mon May 11 03:21:41 UTC 2009  kindle2      yamarama   
5          4   8  Mon May 11 03:22:00 UTC 2009  kindle2  GeorgeVHulme   
6          0   9  Mon May 11 03:22:30 UTC 2009      aig       Seth937   
7          4  10  Mon May 11 03:26:10 UTC 2009   jquery     dcostalis   
8          4  11  Mon May 11 03:27:15 UTC 2009  twitter       PJ_King   
9          4  12  Mon May 11 03:29:20 UTC 2009    obama   mandanicole   

                                                text  
0  @stellargirl I loooooooovvvvvveee my Kindle2. ...  
1  Reading 

In [69]:
training_data.drop(["id", "date", "query", "user"], axis = 1, inplace = True)
test_data.drop(["id", "date", "query", "user"], axis = 1, inplace = True)

In [70]:
print(f"Training data: \n{training_data}\n")
print(f"Test data: \n{test_data}\n")

Training data: 
      sentiment                                               text
0             4  @stellargirl I loooooooovvvvvveee my Kindle2. ...
1             4  Reading my kindle2...  Love it... Lee childs i...
2             4  Ok, first assesment of the #kindle2 ...it fuck...
3             4  @kenburbary You'll love your Kindle2. I've had...
4             4  @mikefish  Fair enough. But i have the Kindle2...
...         ...                                                ...
1987          2  Ask Programming: LaTeX or InDesign?: submitted...
1988          0  On that note, I hate Word. I hate Pages. I hat...
1989          4  Ahhh... back in a *real* text editing environm...
1990          0  Trouble in Iran, I see. Hmm. Iran. Iran so far...
1991          0  Reading the tweets coming out of Iran... The w...

[1992 rows x 2 columns]

Test data: 
     sentiment                                               text
0            4  @stellargirl I loooooooovvvvvveee my Kindle2. ...
1         

In [72]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, 'lxml').get_text()
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
    tweet = re.sub(r"https?://[A-Za-z0-0./]+", ' ', tweet)
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
    tweet = re.sub(r" +", " ", tweet)
    return tweet

column_name = "text"
training_data[column_name] = training_data[column_name].apply(clean_tweet)
test_data[column_name] = test_data[column_name].apply(clean_tweet)
test_data[test_data["label"] == 4] = 1

print(f"Training_data: \n{training_data}\n")
print(f"Test_data: \n{test_data}\n")

C:\Users\PanCh\AppData\Local\Temp\ipykernel_1420\597905739.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  tweet = BeautifulSoup(tweet, 'lxml').get_text()


KeyError: 'label'